In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# 파일 로드

In [12]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("data/diabetes draft7.pdf")

docs = loader.load()
len(docs)

6

# 메타 데이터 확인

In [13]:
docs[5].metadata

{'source': 'data/diabetes draft7.pdf',
 'file_path': 'data/diabetes draft7.pdf',
 'page': 5,
 'total_pages': 6,
 'Author': '@¬8',
 'CreationDate': "D:20240726144349+09'00'",
 'ModDate': "D:20240726144349+09'00'",
 'Producer': 'Microsoft: Print To PDF',
 'Title': 'diabetes draft7.hwp'}

In [14]:
# docs 리스트에 포함된 모든 문서의 metadata 추출
all_metadata = [doc.metadata for doc in docs]

all_metadata

[{'source': 'data/diabetes draft7.pdf',
  'file_path': 'data/diabetes draft7.pdf',
  'page': 0,
  'total_pages': 6,
  'Author': '@¬8',
  'CreationDate': "D:20240726144349+09'00'",
  'ModDate': "D:20240726144349+09'00'",
  'Producer': 'Microsoft: Print To PDF',
  'Title': 'diabetes draft7.hwp'},
 {'source': 'data/diabetes draft7.pdf',
  'file_path': 'data/diabetes draft7.pdf',
  'page': 1,
  'total_pages': 6,
  'Author': '@¬8',
  'CreationDate': "D:20240726144349+09'00'",
  'ModDate': "D:20240726144349+09'00'",
  'Producer': 'Microsoft: Print To PDF',
  'Title': 'diabetes draft7.hwp'},
 {'source': 'data/diabetes draft7.pdf',
  'file_path': 'data/diabetes draft7.pdf',
  'page': 2,
  'total_pages': 6,
  'Author': '@¬8',
  'CreationDate': "D:20240726144349+09'00'",
  'ModDate': "D:20240726144349+09'00'",
  'Producer': 'Microsoft: Print To PDF',
  'Title': 'diabetes draft7.hwp'},
 {'source': 'data/diabetes draft7.pdf',
  'file_path': 'data/diabetes draft7.pdf',
  'page': 3,
  'total_pages':

# 추가할 데이터 생성

GPT에,

이미지를 캡쳐해서 `PDF의 메타데이터에 넣을수 있는 표로 파이썬 코드를 이용해서 작성해줘.` 라고 입력 - 직접 코딩할 일이 없어짐!

In [15]:
import json
table_data = {
    "headers": ["Model", "ROC-AUC"],
    "rows": [
        {"Model": "ADA", "ROC-AUC": 0.51},
        {"Model": "LR", "ROC-AUC": 0.50},
        {"Model": "XGB", "ROC-AUC": 0.51},
        {"Model": "RF", "ROC-AUC": 0.50},
        {"Model": "KNN", "ROC-AUC": 0.50},
        {"Model": "SVC", "ROC-AUC": 0.50},
        {"Model": "BNB", "ROC-AUC": 0.54},
        {"Model": "Ridge", "ROC-AUC": 0.51},
        {"Model": "DT", "ROC-AUC": 0.54},
        {"Model": "GNB", "ROC-AUC": 0.62},
        {"Model": "DNN", "ROC-AUC": 0.81},
        {"Model": "RNN+CNN", "ROC-AUC": 0.83}
    ]
}
table_data_str = json.dumps(table_data)

In [16]:
docs[5].metadata['table'] = table_data_str

In [17]:
docs[5].metadata

{'source': 'data/diabetes draft7.pdf',
 'file_path': 'data/diabetes draft7.pdf',
 'page': 5,
 'total_pages': 6,
 'Author': '@¬8',
 'CreationDate': "D:20240726144349+09'00'",
 'ModDate': "D:20240726144349+09'00'",
 'Producer': 'Microsoft: Print To PDF',
 'Title': 'diabetes draft7.hwp',
 'table': '{"headers": ["Model", "ROC-AUC"], "rows": [{"Model": "ADA", "ROC-AUC": 0.51}, {"Model": "LR", "ROC-AUC": 0.5}, {"Model": "XGB", "ROC-AUC": 0.51}, {"Model": "RF", "ROC-AUC": 0.5}, {"Model": "KNN", "ROC-AUC": 0.5}, {"Model": "SVC", "ROC-AUC": 0.5}, {"Model": "BNB", "ROC-AUC": 0.54}, {"Model": "Ridge", "ROC-AUC": 0.51}, {"Model": "DT", "ROC-AUC": 0.54}, {"Model": "GNB", "ROC-AUC": 0.62}, {"Model": "DNN", "ROC-AUC": 0.81}, {"Model": "RNN+CNN", "ROC-AUC": 0.83}]}'}

# 하던대로 RAG구축

In [54]:
from operator import itemgetter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser


In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

In [22]:
len(documents)

10

In [23]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o-mini")

vector_store = Chroma.from_documents(documents, embeddings)
retriever = vector_store.as_retriever()

# retriever 확인해보기

page_content에는 내용이 들어가 있지 않지만, metadata로 들어간것을 확인할 수 있다.

In [27]:
result = retriever.invoke("knn의 roc-auc값 알려줘")

In [40]:
result

[Document(metadata={'Author': '@¬8', 'CreationDate': "D:20240726144349+09'00'", 'ModDate': "D:20240726144349+09'00'", 'Producer': 'Microsoft: Print To PDF', 'Title': 'diabetes draft7.hwp', 'file_path': 'data/diabetes draft7.pdf', 'page': 2, 'source': 'data/diabetes draft7.pdf', 'total_pages': 6}, page_content='. 기존연구\n. 분석 알고리즘\nFig3.ColumnsCorrelation\nFig2.KNNAlgorithm'),
 Document(metadata={'Author': '@¬8', 'CreationDate': "D:20240726144349+09'00'", 'ModDate': "D:20240726144349+09'00'", 'Producer': 'Microsoft: Print To PDF', 'Title': 'diabetes draft7.hwp', 'file_path': 'data/diabetes draft7.pdf', 'page': 5, 'source': 'data/diabetes draft7.pdf', 'table': '{"headers": ["Model", "ROC-AUC"], "rows": [{"Model": "ADA", "ROC-AUC": 0.51}, {"Model": "LR", "ROC-AUC": 0.5}, {"Model": "XGB", "ROC-AUC": 0.51}, {"Model": "RF", "ROC-AUC": 0.5}, {"Model": "KNN", "ROC-AUC": 0.5}, {"Model": "SVC", "ROC-AUC": 0.5}, {"Model": "BNB", "ROC-AUC": 0.54}, {"Model": "Ridge", "ROC-AUC": 0.51}, {"Model": "DT",

In [41]:
print(result[1].metadata)
print("-------")
print(result[2].metadata)

{'Author': '@¬8', 'CreationDate': "D:20240726144349+09'00'", 'ModDate': "D:20240726144349+09'00'", 'Producer': 'Microsoft: Print To PDF', 'Title': 'diabetes draft7.hwp', 'file_path': 'data/diabetes draft7.pdf', 'page': 5, 'source': 'data/diabetes draft7.pdf', 'table': '{"headers": ["Model", "ROC-AUC"], "rows": [{"Model": "ADA", "ROC-AUC": 0.51}, {"Model": "LR", "ROC-AUC": 0.5}, {"Model": "XGB", "ROC-AUC": 0.51}, {"Model": "RF", "ROC-AUC": 0.5}, {"Model": "KNN", "ROC-AUC": 0.5}, {"Model": "SVC", "ROC-AUC": 0.5}, {"Model": "BNB", "ROC-AUC": 0.54}, {"Model": "Ridge", "ROC-AUC": 0.51}, {"Model": "DT", "ROC-AUC": 0.54}, {"Model": "GNB", "ROC-AUC": 0.62}, {"Model": "DNN", "ROC-AUC": 0.81}, {"Model": "RNN+CNN", "ROC-AUC": 0.83}]}', 'total_pages': 6}
-------
{'Author': '@¬8', 'CreationDate': "D:20240726144349+09'00'", 'ModDate': "D:20240726144349+09'00'", 'Producer': 'Microsoft: Print To PDF', 'Title': 'diabetes draft7.hwp', 'file_path': 'data/diabetes draft7.pdf', 'page': 5, 'source': 'data/d

In [44]:
print(result[1].page_content)
print("-------")
print(result[2].page_content)

Partial Least Squares Path Modeling: Basic Concepts,
Methodological Issues and Applications, pp.183-207, 2023
DOI:10.1007/978-3-031-37772-3_7
[9] N. Shrestha, “Detecting multicollinearity in regression
analysis”, American Journal of Applied Mathematics and
Statistics, vol.8, no.2, pp.39-42, 2020.
DOI:10.12691/ajams-8-2-1
[10]A. Ogunpola, F.Saeed, S. Basurra, A.M. Albarrak, S.N.
Qasem, "Machine learning-based predictive models for
detection of cardiovascular diseases", Diagnostics, vol.14,
2024.DOI:10.3390/diagnostics14020144
김재호(Jae-Ho Kim)
REFERENCES
[1]GojkaRoglic,“WHOGlobalreportindiabetes:Asummary”,
수원대학교컴퓨터학부학사
International Journal ofNoncommunicable Diseases,vol.1,
수원대학교컴퓨터학부석사과정
pp.3-8, Jun.2016.DOI:10.4103/2468-8827.184853 관심분야:인공지능
[2] S. G. Kim, D. S. Choi, “The present State of Diabetes
Mellitus in Korea”, Journal of the Korean Medical
Association, vol.51, pp. 791-798, Sep. 2008. DOI: 김장영(Jang-Young Kim)
https://doi.org/10.5124/jkma.2008.51.9.791
연세대학교컴퓨터과학공학사
-------
https:

In [64]:
template = (
    [
        (
            'system', 
            """
            당신은 도움이 되는 Q&A 챗봇입니다. 주어진 문서에 대해서 질문을 하면 답변을 해주세요.
            답변을 주요 내용과 함께 마크다운 테이블 형식으로 작성해주세요.
            답변의 출처를 적어주세요.
            모르면 모른다고 답변해 주세요. 
            하지만, 당신의 생각을 답변할 수 있습니다. 다만, 당신의 생각이라고 말해야 합니다.
            한국어로 답변해 주세요.

            
            #예제 형식:
            (답변 간략 요약)
            (테이블)
            (질문에 대한 자세한 답변)
            
            **출처**
            - (페이지 소스 및 페이지 번호)
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ('human',
        """
        #Question
        {question}
        #Context
        {context}
        """)

    ]
)
prompt = ChatPromptTemplate.from_messages(template)

In [80]:

llm = ChatOpenAI(model="gpt-4o-mini")

chain = (
{
    "context": itemgetter("question")|retriever,
    "question": itemgetter("question"),
    "chat_history": itemgetter("chat_history"),
}
| prompt
| llm
| StrOutputParser()
)


In [81]:
chain.invoke("본 논문에서 제시하는 ADA모델의 roc-auc 값은 얼마야?")

TypeError: string indices must be integers, not 'str'

In [82]:
store = {}

# 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()

    return store[session_ids]

In [83]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history, # 세션 기록을 가져오는 함수
    input_messages_key="question", # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history" #기록 메세지의 키
)

In [85]:
response = chain_with_history.invoke({"question": "본 논문에서 제시하는 모델들의 roc-auc 값들을 알려줘"},
                           config={"configurable": {"session_id":"abc123"}})

[대화 세션ID]: abc123


In [86]:
print(response)

본 논문에서 제시하는 모델들의 ROC-AUC 값은 다음과 같습니다.

| 모델       | ROC-AUC |
|------------|---------|
| ADA        | 0.51    |
| LR         | 0.50    |
| XGB        | 0.51    |
| RF         | 0.50    |
| KNN        | 0.50    |
| SVC        | 0.50    |
| BNB        | 0.54    |
| Ridge      | 0.51    |
| DT         | 0.54    |
| GNB        | 0.62    |
| DNN        | 0.81    |
| RNN+CNN    | 0.83    |

**출처**
- (페이지 소스: data/diabetes draft7.pdf, 페이지 번호: 5)


In [49]:
chain.invoke("본 논문에서 제시하는 모델들의 roc-auc 값들을 알려줘")

'논문에서 제시하는 모델들의 ROC-AUC 값은 다음과 같습니다:\n\n- ADA: 0.51\n- LR: 0.50\n- XGB: 0.51\n- RF: 0.50\n- KNN: 0.50\n- SVC: 0.50\n- BNB: 0.54\n- Ridge: 0.51\n- DT: 0.54\n- GNB: 0.62\n- DNN: 0.81\n- RNN+CNN: 0.83\n\n출처: "diabetes draft7.pdf" 문서, 페이지 5.'

In [50]:
print(chain.invoke("본 논문에서 제시하는 모델들의 roc-auc 값들을 알려줘"))

본 논문에서 제시하는 모델들의 ROC-AUC 값은 다음과 같습니다:

- ADA: 0.51
- LR: 0.5
- XGB: 0.51
- RF: 0.5
- KNN: 0.5
- SVC: 0.5
- BNB: 0.54
- Ridge: 0.51
- DT: 0.54
- GNB: 0.62
- DNN: 0.81
- RNN+CNN: 0.83

출처: `data/diabetes draft7.pdf`, 페이지 5.


In [51]:
#chain.invoke("knn값은 얼마야?")

'KNN의 ROC-AUC 값은 0.5입니다. \n\n출처: "diabetes draft7.pdf" 문서의 페이지 5.'

In [52]:
#chain.invoke("어떤 모델값에 대한")

'주어진 context에서는 여러 머신러닝 모델의 ROC-AUC 값에 대한 내용이 포함되어 있습니다. 이 표에는 ADA, LR, XGB, RF, KNN, SVC, BNB, Ridge, DT, GNB, DNN, RNN+CNN 모델의 ROC-AUC 값이 나열되어 있습니다. 특히 DNN 모델은 0.81, RNN+CNN 모델은 0.83의 ROC-AUC 값을 가지고 있습니다. \n\n출처: data/diabetes draft7.pdf (page 5)'